En esta fase se llevarán a cabo las transformaciones necesarias para garantizar la calidad y coherencia del
conjunto de datos. Las transformaciones se realizarán mediante funciones de Python aplicadas al
DataFrame.

Algunas de las transformaciones que podrías realizar son:

- Conversión de columnas numéricas que estén almacenadas como texto (price, reviews_per_month,
etc.).
- Limpieza de símbolos o caracteres especiales en columnas como el precio.
- Tratamiento de valores nulos y evaluación de si deben eliminarse, imputarse o mantenerse.
- Detección y análisis de valores duplicados.
- Normalización o estandarización de variables si es necesario.
- Corrección de valores inconsistentes o errores tipográficos en columnas categóricas.
- Eliminación de columnas irrelevantes para el análisis.

In [ ]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [ ]:
df = pd.read_csv('listings_eda.csv', index_col= 0)

In [ ]:
df_listings.head(1)

In [ ]:
def transformacion(df, name):
    
    print(f"Transformacion del dataframe: {name}")
    
    df[['last_rev_year', 'last_rev_month']] = df['last_review'].str.split('-', expand=True).get([0,1])

    df.insert(df.columns.get_loc('last_review')+ 1, 'last_rev_year', df.pop('last_rev_year'))

    df.insert(df.columns.get_loc('last_review')+ 2, 'last_rev_month', df.pop('last_rev_month'))
    
    df['last_rev_year'] = df['last_rev_year'].astype('Int64')

    df['last_rev_month'] = df['last_rev_month'].astype('Int64')
    
    df.drop('last_review', axis=1, inplace=True)
    
    return df

df = transformacion(df, 'listings')

In [ ]:
df.sample()

In [ ]:
def gestion_nulos(df, name):
    
    print(f"Las columnas con nulos de {name} son de tipo: \n")
    print(df[df.columns[df.isna().any()]].dtypes)
    
    col_num_nul = df.columns[df.isna().any()]
    
    for col in col_num_nul:
        print(f"\nDistribución(%) de las categorías (incluyendo nulos) para la columna", col.upper())
        print(f"{(df[col].value_counts(dropna=False, normalize=True)*100).round(2)} %")
        
    '''Teniendo en cuenta el contexto del análisis, el precio es un eje central, clave para:

        - comparar barrios
        - analizar demanda
        - detectar oportunidades
        - estudiar rentabilidad
        - segmentar alojamientos

    Un 24% de nulos en esta variable es un porcentaje altísimo. 
    Imputarlos supondría inventar el precio de unos 6000 alojamientos, 1 de cada 4, 
    pero eliminarlos supone perder el 24% del dataset. 
    
    Vamos a comprobar la distribución del precio antes de tomar una decisión.
    '''

In [ ]:
gestion_nulos(df,'LISTINGS')

In [ ]:
def analisis_precio (df):
    
    print('======= ESTADISTICOS DE PRICE =========')
    
    stats_price = df['price'].describe().round(2)
    print(f"\n{stats_price}")
    
    print('\nVemos precios muy dispersos, con un valor mínimo de 8€ y un valor máximo de 25654€.\n Lo veremos mejor en un boxplot')
    
    plt.boxplot(df['price'].dropna())
    plt.title(f"Boxplot de Precios")
    plt.ylabel('price')
    plt.show()
    
    print('\nEl boxplot muestra una distribución de precios muy asimétrica, con numerosos valores atípicos en el extremo superior. ')
    print('\nEsto indica la presencia de alojamientos con precios extremadamente altos que distorsionan la media.\n Para análisis posteriores sería recomendable aplicar técnicas de tratamiento de outliers como recorte por percentiles o transformación logarítmica.')
    
    print('Vamos a ver cuantos registros nulos en el precio tienen valor 0 en availability')

    # 1) Máscara: price nulo Y disponibilidad 0
    mask_inactivos = df['price'].isna() & (df['availability_365'] == 0)

    # 2) Número de filas inactivas
    apt_inactivos = mask_inactivos.sum()

    # 3) Tasa sobre el total del dataset
    tasa_total = apt_inactivos / df.shape[0] * 100


    print(f"{apt_inactivos} registros tienen PRICE nulo y availability_365 = 0 (apartamentos inactivos).")
    print(f"La tasa de apartamentos inactivos sobre el total de registros del dataset es de {tasa_total.round(2)}%.\nLo más razonable es prescindir de estos registros para evitar sesgos irreales en nuestro análisis.")

    print('Eliminando registros de apartamentos inactivos...')
    
    df = df[~mask_inactivos].copy()
    
    print(f"{apt_inactivos} registros eliminados")
    
    print('Comprobamos los nulos de nuevo')
    
    col_num_nul = df.columns[df.isna().any()]
    
    for col in col_num_nul:
        print(f"\nDistribución(%) de las categorías (incluyendo nulos) para la columna", col.upper())
        print(f"{(df[col].value_counts(dropna=False, normalize=True)*100).round(2)} %")
        
    return df
    

In [ ]:
df = analisis_precio(df)

In [ ]:
def imputacion_nulos(df):
    
    print(f"\nDebido al alto número de outliers imputamos nulos en PRICE usando la mediana")
    print('\nImputando nulos en PRICE...')
          
    df['price'] = df['price'].fillna(df['price'].median())
    
    print(f"\nNulos en PRICE: {df['price'].isna().sum()}")
    
    print('===========')
    
    reviews_col = df[['last_rev_year', 'last_rev_month','reviews_per_month']].columns.tolist()
    
    for col in reviews_col:
        
        print(f"\nPara la imputación de nulos en {col} crearemos un nuevo valor = 0")
        
        print(f"\nImputando nulos en {col}...")
        
        df[col] = df[col].fillna(0)
        
        print(f"\nNulos en {col}: {df['price'].isna().sum()}")
        
        
    return df

In [ ]:
df = imputacion_nulos(df)

In [ ]:
df = df.to_csv('df_clean.csv')

In [ ]:
col_cat

In [ ]:
type(df), df is None

In [ ]:
#SEPARAMOS VARIABLES CATEGÓRICAS Y NUMÉRICAS



print(col_cat_nul)



print(col_num_nul)

### NULOS Y ESTRATEGIAS DE IMPUTACIÓN (categóricos)

- license --> 63% de nulos --> alto --> Nueva Categoría: sin_datos

In [ ]:
#PORCENTAJES DE NULOS CATEGÓRICOS


# Convertimos a porcentaje


### NULOS Y ESTRATEGIAS DE IMPUTACIÓN (numéricos)

- price --> 24% --> alto --> iterative imputer

- last_rev_month --> 20% --> medio-alto --> debe mantenerse

- last_rev_year --> 20% --> medio-alto --> debe manternerse

- reviews_per_month --> 20% --> medio-alto --> debe mantenerse

In [ ]:
# PORCENTAJES DE NULOS NUMÉRICOS

for col in col_num_nul:
    print(f"La distribución de las categorías (incluyendo nulos) para la columna", col.upper())
    display(df[col].value_counts(dropna=False, normalize=True) * 100)  # Convertimos a porcentaje
    print("........................")

#### Teniendo en cuenta el contexto del análisis, el precio es un eje central, clave para:

- comparar barrios
- analizar demanda
- detectar oportunidades
- estudiar rentabilidad
- segmentar alojamientos

#### Un 24% de nulos en esta variable es un porcentaje altísimo. Imputarlos supondría inventar el precio de unos 6000 alojamientos, 1 de cada 4, pero si los elimino pierdo el 24% del dataset. Vamos a comprobar la distribución del precio antes de tomar una decisión. 

### OUTLIERS EN PRICE

In [ ]:
df[col_num_nul].describe().T

#### Vemos precios muy dispersos, con un valor mínimo de 8 y un valor máximo de 25654. 

#### Lo veremos mejor en un boxplot

In [ ]:
for col in list(col_num_nul):
    plt.boxplot(df[col].dropna())
    plt.title(f"Boxplot de {col}")
    plt.ylabel(col)
    plt.show()

### RELACIONES ENTRE VARIABLES PARA DESCUBRIR PATRONES EN LOS NULOS 

In [ ]:
#BUSCAMOS RELACIÓN ENTRE PRECIO , REVIEWS Y AVAILABILITY

print(f"Hay {df['price'].isnull().sum()} nulos en la columna PRICE")

print(f"Hay {df['reviews_per_month'].isnull().sum()} nulos en la columna REVIEWS_PER_MONTH")

print(f"Hay {df[df['availability_365'] == 0].shape[0]} registros con valor 0 en la columna AVAILABILITY_365")

print(f"Hay {df[df['number_of_reviews'] == 0].shape[0]} registros con valor 0 en la columna NUMBER_OF_REVIEWS")

In [ ]:
#VAMOS A VER CUANTOS REGISTROS CON NULOS EN EL PRECIO TIENEN VALOR '0' EN AVAILABILITY

# 1) Máscara: price nulo Y disponibilidad 0
mask_inactivos = df['price'].isna() & (df['availability_365'] == 0)

# 2) Número de filas inactivas
apt_inactivos = mask_inactivos.sum()

# 3) Tasa sobre el total del dataset
tasa_total = apt_inactivos / len(df) * 100


print(f"{apt_inactivos} registros tienen PRICE nulo y availability_365 = 0 (apartamentos inactivos).")
print(f"La tasa de apartamentos inactivos sobre el total de registros del dataset es de {tasa_total.round(2)}%.\nLo más razonable es prescindir de estos registros para evitar sesgos irreales en nuestro análisis.")

tasa_sobre_nulos = apt_inactivos / df['price'].isna().sum() * 100

print(tasa_sobre_nulos)

print(100 - tasa_sobre_nulos)




In [ ]:
df[mask_inactivos].isnull().sum()

In [ ]:
#ENTRE LOS NULOS VEMOS QUE SOLO 1719 REGISTROS TAMBIÉN TINEN NULOS EN REVIEWS. VAMOS A ASEGURANOS DE QUE EL RESTO HAN TENIDO ALGUNA REVIEW 

mask_inactivos_reviews = mask_inactivos & (df['number_of_reviews'] != 0)

print(df.loc[mask_inactivos_reviews, 'number_of_reviews'].count())



##### vamos a eliminar los que no han estado activos nunca 

In [ ]:
mask_inactivos_sin_reviews = mask_inactivos & (df['number_of_reviews'] == 0)

df[mask_inactivos_sin_reviews].shape[0]

df = df[~mask_inactivos_sin_reviews]

df.shape[0]


In [ ]:
for col in col_num_nul:
    display(df[col].value_counts(dropna=False , normalize=True) * 100)

##### HEMOS PASADO DE 24% DE NULOS EN PRICE A 18% Y DE MOMENTO TAN SOLO HEMOS PERDIDO EL 6% DEL DATASET

In [ ]:
df.isnull().sum()

In [ ]:
# 1) Máscara: price nulo Y disponibilidad 0
mask_inactivos = df['price'].isna() & (df['availability_365'] == 0)

# 2) Número de filas inactivas
apt_inactivos = mask_inactivos.sum()

apt_inactivos

In [ ]:
# CUANTOS INACTIVOS CON REVIEWS HAN TENIDO ALGUNA EN EL ÚLTIMO AÑO?

mask_inactivos_reviews_ltm = mask_inactivos & (df['number_of_reviews_ltm'] > 0)
df[mask_inactivos_reviews_ltm].shape[0]

In [ ]:
mask_reviews_antiguas_inactivos = mask_inactivos & (df['price'].isna())

##### Nos quedamos con los apartamentos con precio nulo con fecha de las review a partir de 2020 

In [ ]:
df = df[~mask_reviews_antiguas_inactivos]

In [ ]:
df.shape[0]

In [ ]:
df.loc[df['last_rev_year'] < 2020].shape[0]

In [ ]:
for col in col_nul:
    display(df[col].value_counts(dropna=False, normalize=True).round(2) * 100)

#### IMPUTAMOS NULOS NUMÉRICOS

In [ ]:
# importamos las librerías que necesitamos
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
## ITERATIVE IMPUTER
"""Es una heramienta para imputar valores nulos que, en vez de usar la media o mediana, "predice" lo que puede ir en esos nulos
fijándose en el resto de valores que SÍ tenemos y en los valores que hay en otras columnas.
y cómo esos valores pueden influir en el valor de la columna que intento imputar. Para ello, no hace solo una vez estas predicciones,
si no muchas, para acercarse lo máximo posible a la verdad.

¿Cómo funciona? Intenta encontrar la correlación entre columnas. También se puede usar con valores categóricos.

Pero cuidado, si la columna no tiene relación con las demás variables, si hay más de 40% de nulos o si no hay muchas más columnas,
puede que acabe imputando por un valor sencillo, como la media.

Tiene dos parametros que se usan para controlar su comportamiento.
Max_iter: el número máximo de repeticiones para adivinar los nulos, cuanto más alto, mejor, aunque depende de la cantidad de datos.
(10-50 es bajo 500-1000 es bastante alto)
Random_state: controla la aleatoriedad, me asegura de que cuando lo ejecute varias veces, me de los mismos resultados.
El valor que le demos da igual, mientras lo usemos, se suele usar el 42 (guiño a Guía del autoestopista galáctico)"""



In [ ]:
imputer = IterativeImputer(max_iter = 100, random_state = 42)
df['price'] = imputer.fit_transform(df[['price']])

In [ ]:
df['price'].isnull().sum()

In [ ]:
#METODO FILLNA PARA REVIEWS

reviews_cols = ['reviews_per_month', 'last_rev_year', 'last_rev_month']

df[reviews_cols] = df[reviews_cols].fillna(0)

df[reviews_cols].isnull().sum()



In [ ]:
df['price'] = df['price'].round(2)

df.sample()

In [ ]:
STOP

In [ ]:
df.to_csv('df_clean.csv', index=False)

In [ ]:
#ahora vamos a estudiar las relaciones entre variables de los apartamentos inactivos que han tenido reviews pero no en el último mes

df_reviews_antiguas_precio_nulo = df[~mask_inactivos_reviews_ltm].loc[df['price'].isnull()]

df_reviews_antiguas_precio_nulo.shape[0]



In [ ]:
df_reviews_antiguas_precio_nulo.groupby(['id', 'last_rev_year'])['number_of_reviews'].agg(['count', 'max', 'min']).sort_values('last_rev_year', ascending=False)

In [ ]:
df_reviews_antiguas_precio_nulo.loc[df['last_rev_year'] >= 2020].shape[0]

In [ ]:
df_host_listings_count = df[mask_inactivos_reviews].groupby(['id','last_rev_year','number_of_reviews'])['calculated_host_listings_count'].max().sort_values(ascending=False)

df_host_listings_count.head(10)

In [ ]:
df.columns

In [ ]:
df.isnull().sum()


In [ ]:
# VAMOS A COMPROBAR CUANTOS 'PRICE' NULOS TIENEN 0 REVIEWS

mask_price_reviews = (df['price'].isnull()) & (df['number_of_reviews'] == 0)

df[mask_price_reviews].shape[0]

In [ ]:
# COMPARAMOS LAS COLUMNAS 'PRICE' Y 'REVIEWS_PER_MONTH' para ver en cuantas filas con nulos coinciden
df[['price', 'reviews_per_month']].isna().value_counts()


In [ ]:
# COMPROBAMOS QUE LOS 2019 NULOS QUE COINCIDEN EN PRICE Y REVIEWS PER MONTH, TAMBIÉN COINCIDEN CON VALOR 0 EN LA COLUMMA NUMBER OF REVIEWS

mask_2 = mask & (df['number_of_reviews'] == 0)

print(df[mask_2].shape[0])


In [ ]:
#POR ÚLTIMO VAMOS A COMPROBAR CUANTOS DE ESOS NULOS HAN ESTADO 0 DIAS DISPONIBLES AL AÑO

mask_3 = mask_2 & (df['availability_365'] == 0)

print(f"De los 2019 nulos en 'price' con valor 0 en number of reviews, {df[mask_3].shape[0]} también tienen valor 0 en 'availability'.\nEs decir, son apartamentos que no han estado activos durante este año.")

print('Por lo tanto vamos a eliminar esos registros porque sesgarían nuestro analisis.')

In [ ]:
df = df[~mask_3]


In [ ]:
print(f"Hemos perdido el 6,87 % del Dataset. Vamos a seguir analizando los {df.shape[0]} registros que tenemos.")

In [ ]:
df['price'].isnull().sum()

In [ ]:
print(f"Hay {df['price'].isnull().sum()} nulos en la columna PRICE")

print(f"Hay {df['reviews_per_month'].isnull().sum()} nulos en la columna REVIEWS_PER_MONTH")

print(f"Hay {df[df['availability_365'] == 0].shape[0]} registros con valor 0 en la columna AVAILABILITY_365")

print(f"Hay {df[df['number_of_reviews'] == 0].shape[0]} registros con valor 0 en la columna NUMBER_OF_REVIEWS")

In [ ]:
print(df.columns.get_loc('price'))
print(df.columns.get_loc('availability_365'))
print(df.columns.get_loc('number_of_reviews'))

# O BIEN 

[df.columns.get_loc(col) for col in ['price', 'number_of_reviews', 'availability_365']]


In [ ]:
df[mask].iloc[:,[5, 7, 12]].sample(10)

In [ ]:
df[mask_2].iloc[:, [5, 7, 12]].sample(5)

In [ ]:
import seaborn as sns
sns.heatmap(df[['price', 'reviews_per_month']].isna(), cbar=False)


In [ ]:
df['price'].isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

# Lista de columnas para las que quieres hacer histogramas

# Crear un histograma separado para cada columna
for col in list(col_num_nul):
    plt.figure(figsize=(8, 5))
    plt.hist(df[col].dropna(), bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Histograma de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
df['price'].isnull().sum()


In [ ]:
df['price'] = df['price'].round(2)

In [ ]:
df.sample(10)

In [ ]:
columnas_cat = df.select_dtypes(include = 'O').columns

for col in columnas_cat:
    df[col] = df[col].str.lower()
    
    
#### PROBAR CON RENAME
    
df.head()